In [1]:
# Initialize the environment
using Pkg
Pkg.activate(".")
Pkg.instantiate()
ENV["DATADEPS_ALWAYS_ACCEPT"] = true # accept the download of the MNIST dataset
SHOW_PROGRESS_BAR = true # turn off progress bars

  Activating project at `c:\Users\mkornjaca\Documents\GitHub\BloqadeReservoir.jl\notebooks`


true

In this demo, we process the MNIST dataset with classical spin reservior. The steps should be very familiar to the QRC expert, the only change is the simulation itself that integrates differential equations governing the spin reservoir evolution.

In [14]:
# import required libraries
using MLDatasets
using MultivariateStats
using OneHotArrays
using Flux
using Bloqade
using Colors
using ProgressBars
using JLD2
using LIBSVM
using Statistics
using CSV
using DataFrames
using PythonCall
using PythonPlot

In [3]:
# Download the MNIST dataset

# training data set (each image has 28x28 pixels and there are 60000 training samples)
data_train = MNIST(:train)
# test data set (10000 test samples)
data_test = MNIST(:test) 

dataset MNIST:
  metadata  =>    Dict{String, Any} with 3 entries
  split     =>    :test
  features  =>    28×28×10000 Array{Float32, 3}
  targets   =>    10000-element Vector{Int64}

In [4]:
# Define the struct for CRC layer
Base.@kwdef struct ClassicalDetuningLayer
    atoms # atom positions
    Ω::Real # Rabi frequency
    t_start::Real # evolution starting time
    t_end::Real # evolution ending time
    step::Real  # readout time step
    state::Vector{<:Real} # classical state
    readout::String #readoud ("s" or "ss")
end

ClassicalDetuningLayer

 The CRC is derived by promoting all the qubits (spin $1/2$'s) to $S\rightarrow\infty$, thus making them classical unit vectors ($\hat{S}$).  The dequantized dynamics of the Rydberg Hamiltonian is described by:
\begin{equation}
    \frac{\mathrm{d} \hat{S}_i}{\mathrm{d} t}= \frac{\partial H[\hat{S}]}{\partial \hat{S}_i} \times \hat{S},
\end{equation}
where the effective instantenous magnetic field acting on the $\hat{S}_i$ is:
\begin{equation}
    \frac{\partial H[\hat{S}]}{\partial \hat{S}_i}= \frac{\Omega(t)}{2} \left[\hat{x}\cos{\phi(t)}+\hat{y}\sin{\phi(t)}\right] + \left[-\frac{\Delta_i(t)}{2} + \frac{1}{4} \sum_{j\neq i}V_{ij}\left(1 + \hat{S}_j^{(z)} \right)\right] \hat{z}.
\end{equation}
Thus, the dynamics of $N_q$-classical spins is efficiently simulated by integrating a system of $3N_q$ equations.

In [5]:
function generate_Vmat(locs::Vector{Vector{Float64}}, C6=862690*2π)
    nsites=length(locs)
    Vmat=zeros(nsites,nsites)
    VDel=zeros(nsites)
    for i in 1:nsites
        for j in 1:nsites
            if i != j
                Vmat[i,j]=C6/(norm(locs[i,:]-locs[j,:]))^6
                VDel[i]+=Vmat[i,j]
            end
        end
    end
    return Vmat, VDel
end

function deriv!(du, u, p::Tuple{Int, Vector{<:Real}, <:Real, Matrix{<:Real}, Vector{<:Real}},t)
    #calculating du/dt
    nsites, Delta, Omega, Vmat, VDel =p
    Bv=Vmat*u[3:3:3*nsites]
    for i in 1:nsites
        Bi = [Omega, 0, -Delta[i]+Bv[i]/2+VDel[i]/2]/2
        s = u[3*i-2:3*i]
        du[3*(i-1)+1] = Bi[3]*s[2] - Bi[2]*s[3]
        du[3*(i-1)+2] = Bi[1]*s[3] - Bi[3]*s[1]
        du[3*(i-1)+3] = Bi[2]*s[1] - Bi[1]*s[2] 
    end
    return du
end

function apply_classical_layer(layer::ClassicalDetuningLayer, x::Vector{<:Real})
    # define Rydberg hamiltonian, detunings parameterized in terms of pca values (x)  
    locs=layer.atoms
    readout=layer.readout
    nsites=length(locs)
    Vmat, VDel =generate_Vmat(locs) 
    # At the start of the simulation, all atoms of the system are in ground states
    u0 = layer.state

    t_start = layer.t_start
    t_end = layer.t_end
    t_step = layer.step
    
    # initialize output vector
    steps = floor(Int, (t_end - t_start) / t_step)
    out2=zeros(3*nsites, 3*nsites, steps)
    out3=zeros(nsites, nsites, steps)
    out31=zeros(div(nsites*(nsites-1),2), steps)
    # Numerically simulate the classical evolution with and store the results
    Omega=layer.Ω
    timespan=(t_start,t_end)
    prob = ODEProblem(deriv!,u0,timespan,(nsites,x,Omega,Vmat, VDel))
    sol = solve(prob, RK4(), saveat= t_step, adaptive=false, dt=1e-3, save_start=false)
    sol3=sol[3:3:3*nsites,:]
    if readout=="s"
        out=reduce(vcat, sol)
    elseif readout=="ss"
        for i in 1:steps
            out2[:,:,i]=sol[:,i] .* sol[:,i]'
        end
        out=reduce(vcat,(reduce(vcat,sol),reduce(vcat,out2)))
    elseif readout=="zz"
        for i in 1:steps
            ind=1
            for i1 in 1:nsites
                for i2 in (i1+1):nsites
                    out31[ind, i]= sol3[i1,i]* sol3[i2,i]
                    ind+=1
                end
            end
            #out3[:,:,i]=sol3[:,i] .* sol3[:,i]'
        end
        out=reduce(vcat,(reduce(vcat,sol3),reduce(vcat,out31)))
    elseif readout=="szz"
        ind=1
        for i in 1:steps
            for i1 in 1:nsites
                for i2 in (i1+1):nsites
                    out31[ind, i]= sol3[i1,i]* sol3[i2,i]
                    ind+=1
                end
            end
            #out3[:,:,i]=sol3[:,i] .* sol3[:,i]'
        end
        out=reduce(vcat,(reduce(vcat,sol),reduce(vcat,out3)))
    elseif readout=="z"
        out=reduce(vcat, sol3)
    end
    
    return out
end

# implement functions that apply a `DetuningLayer` to a matrix containing scaled detunings for each image
function apply_classical_layer(layer::ClassicalDetuningLayer, x::Matrix{<:Real})
    iter = SHOW_PROGRESS_BAR ? ProgressBar(1:size(x, 2)) : 1:size(x, 2)
    outs = [apply_classical_layer(layer, x[:, i][:]) for i in iter]
    return hcat(outs...)
end


accuracy(model, xs, targets) = sum(onecold(model(xs), 0:9) .== targets)/length(targets)
function train_linear_nn!(xs_train, ys_train, xs_test, ys_test; 
    regularization::Float64 = 0.0, nepochs::Int = 100, batchsize::Int = 100, 
    opt = Flux.Adam(0.01), verbose::Bool, nonlinear::Bool=false)
    
    model = Chain(
    Dense(length(xs_train[:, 1]), 10),
    softmax
    )

    if nonlinear
        model = Chain(
            Dense(length(xs_train[:, 1]), 100, relu),
            Dense(100, 100, relu),
            Dense(100, 10),
            softmax
        )
    end

    loader = Flux.DataLoader((data = xs_train, label = ys_train); batchsize, shuffle=true);
    ps = Flux.params(model)

    verbose && println("Training...")
    losses = zeros(nepochs)
    accs_train = zeros(nepochs)
    accs_test = zeros(nepochs)
    for epoch in (verbose ? ProgressBar(1:nepochs) : 1:nepochs)
    l = 1.0
    for (x, y) in loader
        if iszero(regularization)
            grads = Flux.gradient(ps) do
                ŷ = model(x)
                l = Flux.crossentropy(ŷ, y)
            end
        else
            grads = Flux.gradient(ps) do
                ŷ = model(x)
                l = Flux.crossentropy(ŷ, y) + regularization * sum(sum(abs, p) for p in ps)
            end
        end
        Flux.update!(opt, ps, grads)
    end
    losses[epoch] = Flux.crossentropy(model(xs_train), ys_train)
    accs_train[epoch] = accuracy(model, xs_train, onecold(ys_train, 0:9))
    accs_test[epoch] = accuracy(model, xs_test, ys_test)
    end
    return losses, accs_train, accs_test
end

function train_svm(xs, ys, test_features, test_targets)
    model = svmtrain(xs, onecold(ys, 0:9); svmtype = SVC, kernel = Kernel.Linear)
    train_ŷ, train_decision_values = svmpredict(model, xs);
    acc_train = mean(train_ŷ .== onecold(ys, 0:9))
    ŷ, decision_values = svmpredict(model, test_features);
    acc_test = mean(ŷ .== test_targets)
    return acc_train, acc_test
end

train_svm (generic function with 1 method)

Let's now implement CRC in a standard reservoir pipeline. Note that in contrast to QRC, simulations with large numbers of spin are possible.

In [19]:
# We first use PCA to downsample the data into 10-dimensional vectors
dim_pca = 8

# Use the `fit` function from the `MultivariateStats` package to generate the projection operator for PCA
model_pca = fit(PCA, reshape(data_train.features, (784, 60000)); maxoutdim=dim_pca);

# Use the `predict` function from `MultivariateStats` package to compute the first 10 principal components
x = MultivariateStats.predict(model_pca, reshape(data_train.features, (784, 60000)))

# Let us see how it looks
num_examples = 10000
xs = x[:, 1:num_examples]

#normalizing features
Δ_max = 6
spectral = max(abs(maximum(xs)), abs(minimum(xs)))
xs = xs/spectral * Δ_max # to make sure values to be between [-6.0, 6.0]

#one hot encoding of labels 
y = onehotbatch(data_train.targets, 0:9)
# Let us see how labels look
ys = y[:, 1:num_examples]

d = 10.0
nsites=dim_pca
locs = [[i*d,0.0] for i in 1:nsites] # put atoms in a chain with 10 micron spacing
u0 =zeros(3*nsites)
for i in 1:nsites
    u0[3*i] = -1 
end
pre_layer = ClassicalDetuningLayer(;
    atoms=locs, 
    Ω = 2π, 
    t_start = 0.0, 
    t_end = 4.0, 
    step = 0.5, 
    state=u0,
    readout="zz"
);

In [20]:
# uncomment the next line to see progress bar
SHOW_PROGRESS_BAR = false 
embeddings = apply_classical_layer(pre_layer, xs)

288×10000 Matrix{Float64}:
 -0.0025347   -0.016056    -0.00667874   …  -0.0200377   -0.0196264
 -0.0394597   -0.037505    -0.00609092      -0.00435202  -0.0030882
 -0.0190444   -0.0621152   -0.0124669       -0.00530816  -0.0586101
 -0.059114    -0.0499508   -0.00379331      -0.0359914   -0.0512428
 -0.021711    -0.0844998   -0.0394741       -0.00952299  -0.0428961
 -0.0341243   -0.0304166   -0.0206318    …  -0.0250836   -0.0117516
 -0.0339379   -0.0181282   -0.00327114      -0.00976858  -0.041323
 -0.0019961   -0.00830646  -0.000632081     -0.0106883   -0.00456156
  0.805169     0.996427     0.656976         0.434101     0.432186
 -0.209162    -0.141048     0.303297         0.368926     0.517966
  ⋮                                      ⋱               
  0.180948    -0.146288     0.623413        -0.0312721    0.00124957
 -0.396762    -0.20815     -0.0549247    …  -0.156231    -0.0291297
  0.425304    -0.181323    -0.155715         0.0326403    0.0386768
  0.00929519  -0.153269    -0.00

In [21]:
num_test_examples = 1000

# using the same PCA model we fit to the test set
test_features = MultivariateStats.predict(model_pca, reshape(data_test.features, (784, 10000)))[:, 1:num_test_examples] 

# same rescaling we applied on train input get reasonable detuning values 
test_features_qrc = test_features/spectral * Δ_max 

# quantum embeddings for 100 test samples
test_embeddings = apply_classical_layer(pre_layer, test_features_qrc)

288×1000 Matrix{Float64}:
 -0.0176704   -0.00440874  -0.0600804   …  -0.00608695  -0.0115501
 -0.00250291  -0.0982701   -0.0476215      -0.00277866  -0.00277022
 -0.0184618   -0.013301    -0.023997       -0.0308231   -0.00327755
 -0.0394076   -0.00606576  -0.00966865     -0.0327782   -0.00718883
 -0.0538571   -0.0231886   -0.0573744      -0.00343375  -0.0184842
 -0.0101416   -0.0516895   -0.0162518   …  -0.020267    -0.0187568
 -0.0454267   -0.0368324   -0.0192229      -0.0704857   -0.0145644
 -0.00109283  -0.00116435  -0.00847021     -0.0066797   -0.000695645
  0.45184      0.811446     0.0902105       0.974704     0.554119
  0.577258    -0.595967    -0.210033        0.468598     0.445898
  ⋮                                     ⋱               
 -0.340072    -0.0216938    0.101647        0.160414     0.588281
 -0.0816118    0.014536     0.143924    …  -0.0423724    0.394969
 -0.530955     0.0572977    0.0661621       0.155324    -0.0879907
  0.422837    -0.0657711   -0.159792        0

And finally, let's train the linear SVM on the CRC embeddings.

In [22]:
tr_svm, ts_svm=train_svm(xs, ys, test_features_qrc, data_test.targets[1:num_test_examples])

(0.8241, 0.797)

In [24]:
tr_svm, ts_svm=train_svm(embeddings, ys, test_embeddings, data_test.targets[1:num_test_examples])

(0.9192, 0.871)

The results are similar to the QRC with the equivalent parameters on this data.